## Misorientation of NC Twins - Topological Model 

### Required Packages

In [1]:
import numpy as np
from numpy.linalg import inv,norm
from nmgtwinning import *
np.set_printoptions(precision=4)

### Metric Tensor & Structure Matrix

In [2]:
## Metric tensor, G
G = [[am**2, am*bm*np.cos(gamma),0],[am*bm*np.cos(gamma),bm**2,0],[0,0,cm**2]]

# V₁ lattice vectors in reference frame
ar = [am,0,0]
br = [bm*np.cos(gamma),bm*np.sin(gamma),0]
cr = [0,0,cm]

## ᵣPₘ
rPm = np.column_stack((ar,br,cr))
del ar,br,cr

### Equilibrium Tilt

#### Burgers vector of compound twins

$V_1 \to V_3: (010)_{TB} =(\mathbf{b}_{13},h_{13})$

$V_3 \to V_4: (110)_{TB} = (\mathbf{b}_{34},h_{34})$

In [3]:
## obtain burgers vector and step height

#(010) TB
tmu = rPm @ [0,1/2,1/2]
R = rotationmatrix([0,1,0] @ inv(rPm),np.pi)
tlamda = R @ rPm @ [0,1/2,-1/2]
b13 = norm(tlamda - tmu)
h13 = 1/np.sqrt([0,2,0] @ inv(G) @ [0,2,0])
del tmu,R,tlamda

#(110) TB
tmu = rPm @ [1/2,0,1/2]
R = rotationmatrix([1,1,0] @ inv(rPm),np.pi)
tlamda = R @ rPm @ [0,1/2,-1/2]
b34 = norm(tlamda - tmu)
h34 = 1/np.sqrt([2,2,0] @ inv(G) @ [2,2,0])
del tmu,R,tlamda

#print disconnections properties
print('b₁₃ = %6.4f, h₁₃ = %6.4f, b₃₄ = %6.4f, h₃₄ = %6.4f' % (b13,h13,b34,h34)) 

## minimization constant
k = (b13*h34)/(b34*h13)
print('k = (b₁₃ h₃₄) / (b₃₄ h₁₃) = %6.3f' % k)

b₁₃ = 0.0038, h₁₃ = 0.2972, b₃₄ = 0.0020, h₃₄ = 0.2100
k = (b₁₃ h₃₄) / (b₃₄ h₁₃) =  1.374


#### Angle between $(\bar{1} \bar{1} 0)$ and $(0 \bar{1} 0)$ plane

In [4]:
# Angle between (-1,-1,0) and (0,-1,0) plane
adotb = [-1,-1,0] @ inv(G) @ [0,-1,0]
a = np.sqrt([-1,-1,0] @ inv(G) @ [-1,-1,0])
b = np.sqrt([0,-1,0] @ inv(G) @ [0,-1,0])
psi = np.arccos(adotb/(a*b))
del a,b,adotb
print('ψ = %4.3f°' % np.degrees(psi)) 

ψ = 44.681°


#### Strain Equilibrium

We obtain the equilibrium misorientation $\theta$ at which the strain along the interface vanishes when $(\mathbf{b}_{13},h_{13})$ and $(\mathbf{b}_{34},h_{34})$ disconnections meet along a interface.

In [5]:
## Numerically calculate theta
## Minimization equation: sin 2(ψ − θ) - k sin 2θ = 0

## range of theta from 0 to π/2
theta = np.linspace(0, np.pi/2, 10000)

## obtain the equation value
eqn = []
for i in theta:
    dummy = np.abs(np.sin(2*(psi-i)) - k*np.sin(2*i))
    eqn.append(dummy)
    del dummy

## find the equation minimum
min_index = np.where(eqn == np.min(eqn)) # index with closest 0 value
theta = theta[min_index[0][0]] # theta for zero solution
print('θ = %4.3f°' % np.degrees(theta)) # print theta
del eqn, min_index

θ = 17.912°


#### Additional misorientation due to Perpendicular Burgers' vector component

In [6]:
phi = b13*np.sin(theta)**2/h13
print('ϕ = %4.3f°' % np.degrees(phi))
phi_p = b34*np.sin(psi - theta)**2/h13
print('ϕ\' = %4.3f°' % np.degrees(phi_p))
Delta = phi_p + phi
print ('ϕ(avg) = %4.3f°' % (np.degrees(Delta)/2))
print('Δ = ϕ + ϕ\' = %4.4f°' % np.degrees(Delta))
print('θ + Δ/2 = %4.2f°' % np.degrees(theta + Delta/2)) 
del phi,phi_p,b13,h13,b34,h34

ϕ = 0.070°
ϕ' = 0.077°
ϕ(avg) = 0.074°
Δ = ϕ + ϕ' = 0.1472°
θ + Δ/2 = 17.99°


### Misorientation Prediction using TM

#### TB in terms of V1

In [7]:
## Calculate parameters using sine rule
alpha = np.pi - theta - (np.pi - gamma)
print('α = %4.2f°' % np.degrees(alpha))
bstar_1 = np.sin(theta)/np.sin(alpha) * am
print('b*₁ = %4.4f nm' % bstar_1) 
print('b*₁ / bₘ = %4.4f' % (bstar_1/bm))
print()

## [AB]ₘ₁ vector
AB_vec_1 = np.asarray([1,bstar_1/bm,0])
print('[AB]ₘ₁ = [', round(AB_vec_1[0],4),', ', round(AB_vec_1[1],4),', ', round(AB_vec_1[2],4),']ₘ₁')

## Magnitude of AB
L_star1 = np.sqrt(AB_vec_1 @ G @ AB_vec_1)
AB_plane_1 = np.asarray([-1,bm/bstar_1,0])
print('L*₁ = %.4f nm' % L_star1)
print()

## (AB)ₘ₁ plane
print('(AB)ₘ₁ = (', round(AB_plane_1[0],4),', ', round(AB_plane_1[1],4),', ', round(AB_plane_1[2],4),')ₘ₁')
del alpha,bstar_1

α = 72.46°
b*₁ = 0.1926 nm
b*₁ / bₘ = 0.3241

[AB]ₘ₁ = [ 1.0 ,  0.3241 ,  0.0 ]ₘ₁
L*₁ = 0.6263 nm

(AB)ₘ₁ = ( -1.0 ,  3.0857 ,  0.0 )ₘ₁


#### TB in terms of $V_4$

In [8]:
# Remaining angle of the triangle
phi = np.arcsin(bm/L_star1 * np.sin(gamma))
print('ϕ = %4.2f°' % np.degrees(phi))
epsilon = np.pi - gamma - phi
print('ε = %4.2f°' % np.degrees(epsilon))
print()

# Magnitude of a*
astar_4 = np.sin(epsilon)/np.sin(gamma) * L_star1
print('a*₄ = %4.4f nm' % astar_4)
print('a*₄ / aₘ = %4.4f' % (astar_4/am))
print()

## [AB]ₘ₄ vector
AB_vec_4 = np.array([-astar_4/am,1,0])
print('[AB]ₘ₄ = [', round(AB_vec_4[0],4),', ', round(AB_vec_4[1],4),', ', round(AB_vec_4[2],4),']ₘ₄')

## Magnitude of AB
L_star4 = np.sqrt(AB_vec_4 @ G @ AB_vec_4)
print('L*₄ = %.4f nm' % L_star4)

## (AB)ₘ₄ plane
AB_plane_4 = np.asarray([-am/astar_4, -1, 0])
print('(AB)ₘ₄ = (', round(AB_plane_4[0],4),', ', round(AB_plane_4[1],4),', ', round(AB_plane_4[2],4),')ₘ₄')
print()

del phi, epsilon, astar_4, L_star1, L_star4

ϕ = 71.63°
ε = 18.00°

a*₄ = 0.1936 nm
a*₄ / aₘ = 0.3241

[AB]ₘ₄ = [ -0.3241 ,  1.0 ,  0.0 ]ₘ₄
L*₄ = 0.6263 nm
(AB)ₘ₄ = ( -3.0852 ,  -1.0 ,  0.0 )ₘ₄



#### TM Misorientation Prediction

In [9]:
## Angle between AB plane in V1 and V4
adotb = AB_plane_1 @ inv(G) @ -AB_plane_4
a = np.sqrt(AB_plane_1 @ inv(G) @ AB_plane_1)
b = np.sqrt(-AB_plane_4 @ inv(G) @ -AB_plane_4)
ang_ab = np.arccos(adotb/(a*b))
TM_mis = ang_ab + Delta
print('Angle between (AB)₁ & (AB)₄ = %4.2f°' % (np.degrees(ang_ab)))
print('Misorientation using TM - L₁₋₄ = %4.2f°' % (np.degrees(TM_mis)))
del adotb,a,b, ang_ab

Angle between (AB)₁ & (AB)₄ = 89.54°
Misorientation using TM - L₁₋₄ = 89.69°


#### Obtain Equilibrium Indicies

In [10]:
## Obtain q_TM
L = rotationmatrix([0,0,+1],Delta/2) # rotation matrix
AB_1 = inv(rPm) @ L @ rPm @ AB_vec_1 # rotate vector
print('counterclockwise rotated [AB]ₘ₁ = [', round(AB_1[0],4),', ', round(AB_1[1],4),', ', round(AB_1[2],4),']ₘ₁')
q_TM = -AB_1[0] / AB_1[1]
print('q(TM) = %4.4f' % q_TM)
del L, AB_1

counterclockwise rotated [AB]ₘ₁ = [ 0.9996 ,  0.3254 ,  0.0 ]ₘ₁
q(TM) = -3.0723


In [11]:
## Obtain r_TM
L = rotationmatrix([0,0,-1],Delta/2)
AB_4 = inv(rPm) @ L @ rPm @ AB_vec_4
print('clockwise rotated [AB]ₘ₄ = [', round(AB_4[0],4),', ', round(AB_4[1],4),', ', round(AB_4[2],4),']ₘ₄')
r_TM = -AB_4[0] / AB_4[1]
print('r(TM) = %4.4f' % r_TM)
del L, AB_4

clockwise rotated [AB]ₘ₄ = [ -0.3229 ,  1.0004 ,  0.0 ]ₘ₄
r(TM) = 0.3227


In [12]:
## Obtain angle between [-1,-q_TM,0] and [1,r_TM,0]
a = [-1,-q_TM,0]; b = [1,r_TM,0]
adotb = a @ inv(G) @ b
a = np.sqrt(a @ inv(G) @ a)
b = np.sqrt(b @ inv(G) @ b)
ang_ab = np.arccos(adotb/(a*b))
print('Angle between (-1, -q(TM), 0) & (1, r(TM), 0) = %4.2f°' % (np.degrees(ang_ab)))
del a, b, adotb, ang_ab

Angle between (-1, -q(TM), 0) & (1, r(TM), 0) = 89.69°


### TJL Disclination strength according to TM

In [13]:
## θ₁
theta1 = angleTBpair(rPm, [1, q_TM, 0], [0, 1, 0])
print('θ₁ = %4.2f°' % round(np.degrees(theta1),5))

## θ₃
theta3 = angleTBpair(rPm, [0, -1, 0], [-1, -1, 0])
print('θ₃ = %4.2f°' % round(np.degrees(theta3),5))

## θ₄
theta4 = angleTBpair(rPm, [-1, -1, 0], [1, r_TM, 0])
print('θ₄ = %4.2f°' % round(np.degrees(theta4),5))


## sum of angles
print('θ₁ + θ₃ + θ₄ = %4.2f°' % np.degrees(theta1 + theta3 + theta4))

##dislination
omega134 = 2*np.pi - theta1 - theta3 - theta4
print('ω = %.3f°' % np.degrees(omega134))
del theta1, theta3, theta4

θ₁ = 162.01°
θ₂ = 44.68°
θ₃ = 152.98°
θ₁ + θ₃ + θ₄ = 359.68°
ω = 0.324°


In [13]:
## For the notebook '04. TM_NCTwins_Disconnection'
print(q_TM)
print(TM_mis)

-3.072278605191688
1.565322459596643
